In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%time
import zipfile
for i in [0, 1, 2, 3]:
    with zipfile.ZipFile(f'drive/MyDrive/OffroadSegmentation/data/train_images_A_{i}.zip') as existing_zip:
        existing_zip.extractall('train_images')

with zipfile.ZipFile(f'drive/MyDrive/OffroadSegmentation/data/train_annotations_A.zip') as existing_zip:
    existing_zip.extractall('train_annotations')

!cp -r drive/MyDrive/OffroadSegmentation/data/precision_test_images precision_test_images

!pip install segmentation-models-pytorch 
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir

In [ ]:
!pip install -U git+https://github.com/qubvel/segmentation_models.pytorch --no-cache-dir

In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob

from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset

import torch
import numpy as np
import segmentation_models_pytorch as smp

import albumentations as albu
from tqdm import tqdm

from requests import get
filename = get('http://172.28.0.2:9000/api/sessions').json()[0]['name'].split('.')[0]

df = pd.read_csv('/content//drive/MyDrive/Signate-OffroadSegmentation/data/5fold_validation.csv')
df['file_name'] = df['png_name']

fold = int(filename.split('fold')[1])
t_df = df[df['type'] == 'train_images']
train_df = t_df[(t_df['nunique_type'] == 2) |
                ((t_df['fold'] != fold) & (t_df['nunique_type'] == 1))]

valid_df = t_df[(t_df['fold'] == fold) & (t_df['nunique_type'] == 1)]

In [ ]:
import random
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed()

In [ ]:
class Dataset(BaseDataset):
    """CamVid Dataset. Read images, apply augmentation and preprocessing transformations.
    
    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    """
    
    
    def __init__(
            self, 
            df, 
            augmentation=None, 
            preprocessing=None,
    ):
        self.images_fps = df['file_name'].values
        self.masks_fps = df['annotation'].values
                
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        
        # read data
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.masks_fps[i])
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)

        anno_list = [[128, 64, 128], [255, 128, 128], [0, 0, 70]]  # road, dirt road, other obstacle
        masks = [((mask[:, :, 0] == a[0]) & (mask[:, :, 1] == a[1]) & (mask[:, :, 2] == a[2])) for a in anno_list]
        mask = np.stack(masks, axis=-1).astype('float')

        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
            
        return image, mask
        
    def __len__(self):
        return len(self.images_fps)

In [ ]:

def get_training_augmentation():
    IMAGE_SIZE = [1080, 1920]
    train_transform = [
        albu.Resize(*[1056, 1920]),
        albu.PadIfNeeded(1056, 1920),
        

        # albu.HorizontalFlip(p=0.5),

        # albu.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=0.8, border_mode=0),

        # albu.PadIfNeeded(min_height=1080, min_width=1920, always_apply=True, border_mode=0),
        # albu.RandomCrop(height=960, width=960, always_apply=True),
        # albu.RandomCrop(height=IMAGE_SIZE[0], width=IMAGE_SIZE[0], always_apply=True),

        # albu.IAAAdditiveGaussianNoise(p=0.2),
        # albu.IAAPerspective(p=0.5),

        # albu.OneOf(
        #     [
        #         albu.CLAHE(p=1),
        #         albu.RandomBrightness(p=1),
        #         albu.RandomGamma(p=1),
        #     ],
        #     p=0.9,
        # ),

        # albu.OneOf(
        #     [
        #         albu.IAASharpen(p=1),
        #         albu.Blur(blur_limit=3, p=1),
        #         albu.MotionBlur(blur_limit=3, p=1),
        #     ],
        #     p=0.9,
        # ),

        # albu.OneOf(
        #     [
        #         albu.RandomContrast(p=1),
        #         albu.HueSaturationValue(p=1),
        #     ],
        #     p=0.9,
        # ),
    ]
    return albu.Compose(train_transform)


def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    IMAGE_SIZE = [1080, 1920]
    test_transform = [
        albu.Resize(*[1056, 1920]),
        albu.PadIfNeeded(1056, 1920)
    ]
    return albu.Compose(test_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

In [ ]:
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()

augmented_dataset = Dataset(
    train_df,
    augmentation=get_training_augmentation(), 
)


# same image with different random transforms
for i in range(3):
    image, mask = augmented_dataset[i]
    visualize(image=image, mask=mask)

In [ ]:
ENCODER = 'resnet18'
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'softmax2d' # could be None for logits or 'softmax2d' for multicalss segmentation
DEVICE = 'cuda'

# create segmentation model with pretrained encoder
model = smp.Unet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    in_channels=3,
    classes=3, 
    activation=ACTIVATION,
)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

train_dataset = Dataset(
    train_df, 
    augmentation=get_training_augmentation(), 
    preprocessing=get_preprocessing(preprocessing_fn),
)

valid_dataset = Dataset(
    valid_df, 
    augmentation=get_validation_augmentation(), 
    preprocessing=get_preprocessing(preprocessing_fn),
)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=12)
valid_loader = DataLoader(valid_dataset, batch_size=4, shuffle=False, num_workers=4)

loss = smp.losses.FocalLoss('multilabel')
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.0001),
])

In [ ]:
# focal loss, resizemix を使うためにsmpをすこし変更

import sys
import torch
from tqdm import tqdm as tqdm
import torch.nn.functional as F
from segmentation_models_pytorch.utils.meter import AverageValueMeter

def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

def resizeMix(data, targets, alpha=1):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets1 = targets[indices]

    lam = np.random.beta(alpha, alpha)
    bbx1, bby1, bbx2, bby2 = rand_bbox(data.size(), lam)

    resize_data = F.interpolate(data, [bbx2-bbx1, bby2-bby1])
    resize_targets = F.interpolate(targets, [bbx2-bbx1, bby2-bby1])
    data[:, :, bbx1:bbx2, bby1:bby2] = resize_data[indices, :, :, :]
    targets[:, :, bbx1:bbx2, bby1:bby2] = resize_targets[indices, :, :, :]

    return data, targets


class Epoch:

    def __init__(self, model, loss, metrics, stage_name, device='cpu', verbose=True):
        self.model = model
        self.loss = loss
        self.metrics = metrics
        self.stage_name = stage_name
        self.verbose = verbose
        self.device = device

        self._to_device()

    def _to_device(self):
        self.model.to(self.device)
        self.loss.to(self.device)
        for metric in self.metrics:
            metric.to(self.device)

    def _format_logs(self, logs):
        str_logs = ['{} - {:.4}'.format(k, v) for k, v in logs.items()]
        s = ', '.join(str_logs)
        return s

    def batch_update(self, x, y):
        raise NotImplementedError

    def on_epoch_start(self):
        pass

    def run(self, dataloader, n_epoch, epoch, train=True):

        self.on_epoch_start()

        logs = {}
        loss_meter = AverageValueMeter()
        metrics_meters = {metric.__name__: AverageValueMeter() for metric in self.metrics}

        with tqdm(dataloader, desc=self.stage_name, file=sys.stdout, disable=not (self.verbose)) as iterator:
            for x, y in iterator:
                x, y = x.to(self.device), y.to(self.device)

                if (train) & (n_epoch - epoch) > 10:  # 最後の10回はresizemixなしで学習する
                    x, y = resizeMix(x, y)

                loss, y_pred = self.batch_update(x, y)

                # update loss logs
                loss_value = loss.cpu().detach().numpy()
                loss_meter.add(loss_value)
                # loss_logs = {self.loss.__name__: loss_meter.mean}
                loss_logs = {'FocalLoss': loss_meter.mean}
                logs.update(loss_logs)

                # update metrics logs
                for metric_fn in self.metrics:
                    metric_value = metric_fn(y_pred, y).cpu().detach().numpy()
                    metrics_meters[metric_fn.__name__].add(metric_value)
                metrics_logs = {k: v.mean for k, v in metrics_meters.items()}
                logs.update(metrics_logs)

                if self.verbose:
                    s = self._format_logs(logs)
                    iterator.set_postfix_str(s)

        return logs


class TrainEpoch(Epoch):

    def __init__(self, model, loss, metrics, optimizer, device='cpu', verbose=True):
        super().__init__(
            model=model,
            loss=loss,
            metrics=metrics,
            stage_name='train',
            device=device,
            verbose=verbose,
        )
        self.optimizer = optimizer

    def on_epoch_start(self):
        self.model.train()

    def batch_update(self, x, y):
        self.optimizer.zero_grad()
        prediction = self.model.forward(x)
        loss = self.loss(prediction, y)
        loss.backward()
        self.optimizer.step()
        return loss, prediction


class ValidEpoch(Epoch):

    def __init__(self, model, loss, metrics, device='cpu', verbose=True):
        super().__init__(
            model=model,
            loss=loss,
            metrics=metrics,
            stage_name='valid',
            device=device,
            verbose=verbose,
        )

    def on_epoch_start(self):
        self.model.eval()

    def batch_update(self, x, y):
        with torch.no_grad():
            prediction = self.model.forward(x)
            loss = self.loss(prediction, y)
        return loss, prediction

In [ ]:
train_epoch = TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

In [ ]:
n_epoch = 50
max_score = 0
model_path = f'/content//drive/MyDrive/OffroadSegmentation/model/{filename}.pth'

for i in range(0,50):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader, n_epoch, i, train=True)
    valid_logs = valid_epoch.run(valid_loader, n_epoch, i, train=False)

    # do something (save model, change lr, etc.)
    if valid_logs['iou_score'] > max_score:
        max_score = valid_logs['iou_score']
        torch.save(model, model_path)
        print('Model saved!')
        
    if i == 10:
        optimizer.param_groups[0]['lr'] = 1e-5
        print('Decrease decoder learning rate to 1e-5!')


In [ ]:
valid_logs

In [ ]:
torch.save(model, model_path)